In [1]:
import pandas as pd

### Importing CSV files and drop unnecessary fields

In [2]:
data = pd.read_csv("24_1.csv")
df = pd.DataFrame(data)
df.drop(['Join Time',"Leave Time","Duration","Email"],axis=1,inplace=True)

In [3]:
new_col = df["Role"].str.split("m ")
for i in range(len(new_col)):
    if len(new_col[i]) > 1:
        new_col[i] = int(new_col[i][:-1].pop(-1))
    else:
        new_col[i] = 0
new_df = pd.DataFrame({"merged_time":new_col})    #merging time according Role Fields as given

In [4]:
df = pd.concat([df,new_df],axis="columns")
df.drop(['Role'],axis=1,inplace=True)

# Aggregating the functions 

In [5]:
aggregation_functions = {'Full Name': 'first', 'Participant ID (UPN)': 'first','merged_time': 'sum'}
final_report = df.groupby(df["Full Name"]).aggregate(aggregation_functions)

In [6]:
print(final_report.shape)

(80, 3)


In [7]:
# for creating index list for adding extra column
index_list = [x for x in range(1,len(final_report.index)+1)]

# Adding index to the final report list
final_report["index"] = index_list


### Logic Based on merged_time and merge into last report

In [8]:
check_list = []
for i in final_report["merged_time"]:
    if(i > 20):
        check_list.append("PRESENT")
    else:
        check_list.append("ABSENT")
        
check_list_merger = pd.DataFrame({"ATTENDANCE":check_list,"index":index_list})

In [9]:
final_report = final_report.merge(check_list_merger, how='inner', on='index')
final_report.drop(["index","merged_time"],axis=1,inplace=True)

In [10]:
final_report.to_csv("output.csv",index=False)